Example notebook for client-server connection and simple measurement. It simply generates a wave with some specified parameters and reads it for a determined amount of time.

In [ ]:
from client import mylogger
from instrument import Marcj
import time
import numpy as np
import matplotlib.pyplot as plt

In [2]:
inst = Marcj("JJ")

In [3]:
inst.connect()

In [4]:
amplitude = inst.set_amplitude(2)
freq = inst.set_frequency(300)
wave = inst.set_wave('TRIA')
name = inst.get_id()
#offset = inst.set_offset(0)
#duration = inst.set_duration(3)
#mylogger.info(offset)

In [5]:
play = inst.play('CONT')
mylogger.info(play)

In [6]:
time.sleep(1)

In [7]:
acq_time = 1
nloops = inst.read_signal(acq_time)

In [8]:
ch0 = nloops["ch0"]
ch1 = nloops["ch1"]


ch0 = np.array(ch0)*2.96/inst.maxval
ch1 = np.array(ch1)*2.96/inst.maxval
t = np.linspace(0, acq_time, len(ch0))

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(15,5))
ax1.plot(t, ch0, 'o', markersize=.5, color = "darkblue", label = "resistor")
ax2.plot(t, ch1, 'o', markersize=.5, color = "darkorange", label = "diode+resistor")
fig.suptitle("Signal")
ax1.grid(alpha = .2)
ax2.grid(alpha = .2)
ax1.legend(loc = "best")
ax2.legend(loc = "best")
ax1.set_ylabel("amplitude CH0 [V]")
ax1.set_xlabel("time [s]")
ax2.set_ylabel("amplitude CH1 [V]")
ax2.set_xlabel("time [s]")
ax1.set_xlim(0,0.005);
ax2.set_xlim(0,0.005);

In [ ]:
fig, ax1 = plt.subplots()
color = "darkblue"
ax1.set_xlabel("time [s]")
ax1.set_ylabel("amplitude [V]", color = color)
ax1.plot(t, ch0, 'o', markersize=.5, color = color, label = "diode + resistor")
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = "darkorange"
ax2.set_xlabel("time [s]")
ax2.set_ylabel("amplitude [V]", color = color)
ax2.plot(t, ch1, 'o', markersize=.5, color = color, label = "resistor")
ax2.tick_params(axis='y', labelcolor=color)

ax1.set_xlim(0,0.005);
ax1.grid(alpha=.2)
ax2.set_xlim(0,0.005);
ax2.grid(alpha=.2)

fig.tight_layout() 
plt.show()

In [ ]:
plt.plot(t, ch0, 'o', markersize=.5, color = "darkblue", label = "diode + resistor")
plt.plot(t, ch1, 'o', markersize=.5, color = "darkorange", label = "resistor")
plt.grid(alpha = .2)
plt.legend(loc = "best")
plt.ylabel("amplitude [V]")
plt.xlabel("time [s]")
plt.xlim(0,0.005);

In [ ]:
I_dut = ch0/1000
V_dut = ch1-ch0

plt.plot(V_dut, I_dut, 'o', markersize=.5, color = "darkblue")
plt.grid(alpha = .2)
plt.title("IV diode 1N4007")
plt.ylabel("current [A]")
plt.xlabel("voltage [V]");
#plt.xlim(0,0.002);

In [ ]:
print(len(ch0))

In [ ]:
from scipy.fft import rfft, rfftfreq
import numpy as np

yft = rfft(ch0)
xft = rfftfreq(int(len(ch0)), 1/192000)
spectrum = 20*np.log10(np.abs(yft)/np.max(np.abs(yft)))

#yft_2 = rfft(ch1)
#xft_2 = rfftfreq(int(len(ch1)), 1/192000)
#spectrum_2 = 20*np.log10(np.abs(yft_2)/np.max(np.abs(yft_2)))

plt.plot(xft, spectrum, 'o', markersize=.5, color = "darkblue")
#plt.plot(xft_2, spectrum_2, 'o', markersize=.5, color = "darkorange")
plt.xscale('log');
plt.title("Signal spectrum")
plt.grid(alpha = .2)
plt.ylabel("Intensity [dB]")
plt.xlabel("frequency [Hz]")
xft[np.argmax(spectrum)]

In [15]:
time.sleep(5)

In [16]:
stop = inst.stop_signal()
mylogger.info(stop)

Let's do some studies and processing on this signal

In [ ]:
max_index = []
max_values = []

for i, f in enumerate(spectrum):
    if f > -10:
        max_index.append(i)
        max_values.append(f)

signal_freq = xft[np.argmax(spectrum)]
label_text = str(signal_freq) + " Hz"

plt.plot(xft[max_index], max_values, "x", color = "darkorange", label = label_text)
plt.plot(xft, spectrum, markersize=.5, color = "darkblue", linewidth = .7, label = "Spectrum")
plt.title("Signal spectrum")
plt.legend(loc = "best")
plt.grid(alpha = .2)
plt.ylabel("Intensity [dB]")
plt.xlabel("frequency [Hz]")
plt.xscale('log');

print(str(signal_freq) + " Hz")
print(xft[max_index])

In [18]:
inst.disconnect()